# Исследование надежности заемщиков

Целью данного проекта является **исследование влияния различных факторов на факт погашения кредита в срок** по статистике о платежеспособности. В ходе работы проведем предварительную обработку данных, выявим аномалии, проведем оптимимзацию. Исследуем влияние различных параметров на исследуемую величиную. 

## 1. Исследование и предобработка

In [5]:
import pandas as pd

In [6]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [7]:
# Вывод на экран первых 20 значений
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [8]:
# Вывод основной информации о датафрейме 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проведем предобработку исследуемого датафрейма: выявим пропуски, аномальные значения и тд. 

In [9]:
# Исследуем количество пропущенных значений
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обработаем пропуски данных - на сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце  медианным значением по каждому типу из столбца income_type

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Аномальные отрицательные значения количества дней работы заменим значением по модулю. 

In [11]:
data['days_employed'] = data['days_employed'].abs()

Рассчитаем медианные значения дней трудоустройства по типу дохода. 

In [12]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. 

Рассмотрим уникальные значения столбца children. 

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Удалим аномальные - отрицательные значения. 

In [14]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Пропуски в столбце days_employed также заменим медианными значениями в соответствии с типом. 

In [15]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что пропуски заполнены 

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Оптимизируем тип данных - изменим вещественный на целочисленный. 

In [17]:
data['total_income'] = data['total_income'].astype(int)

Обработаем неявные дубликаты в графе образование. 

In [18]:
data['education'] = data['education'].str.lower()

Остальные дубликаты удалим.

In [19]:
data.duplicated().sum()

71

In [20]:
data = data.drop_duplicates()

## 2. Категоризация данных 

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [21]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [22]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Рассмотри цели взятия кредита 

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [24]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [25]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. Исследование зависимостей 


Для исследования зависимости между количеством детей и возможностью вернуть кредит в срок сгруппируем исследуемые данные по количеству детей. Определим общее количество кредитополучателей с определенным количеством детей. В каждой группе также посчитаем количество долгов. После этого рассчитаем какую часть составляют задолжники в в каждой группе от общего числа. 

In [26]:
# Определим вспомогательную функцию для посчета общего числа людей в группах, и процентное соотношение должников 
def summary(x):
    result = {
        'number_of_person': x['debt'].sum(),
        'mean_debt': 100*x['debt'].mean(),
    }
    return pd.Series(result).round(2)

#cгруппируем полученные данные и выведем в виде свободной таблицы
data.groupby(['children']).apply(summary)

,number_of_person,mean_debt
children,,
0,1063.0,7.54
1,444.0,9.23
2,194.0,9.45
3,27.0,8.18
4,4.0,9.76
5,0.0,0.00


**Вывод:**
По полученным данным *нельзя установить однозначную взаимосвязь между количеством детей и способностью вернуть кредит*. Зависимость носит нелинейный характер и утверждать, что чем больше детей, тем больше должников по представленным данным нельзя. Видно, что при отсутсвии детей вероятность вернуть кредит меньше, чем при наличии 1, 2, 3 или 4 детей.  Следует отметить, что размеры выборок для каждой категории слишком разные (имеют разные порядки) и, к примеру, количество людей, имеющих 5 детей, значительно меньше всех остальных категорий, поэтому проводить сравнение с другими категориями не совсем корректно.

In [27]:
# Аналогично проделаем для семейного статуса
data.groupby(['family_status']).apply(summary)

,number_of_person,mean_debt
family_status,,
Не женат / не замужем,273.0,9.76
в разводе,84.0,7.06
вдовец / вдова,63.0,6.62
гражданский брак,385.0,9.31
женат / замужем,927.0,7.56


**Вывод:** 
Из полученных данных можно сделать вывод, что *не женатые/не замужние и состоящие в гражданском браке чаще не возвращают кредиты*, чем разведенные, вдовы/вдовцы и замужние/женатые.  

Проведем аналогичную группировку данных по ранее введенной классификации уровня дохода и рачитаем процентное соотношение должников.


In [29]:
# Аналогично проделаем для разных уровней дохода
data.groupby(['total_income_category']).apply(summary)

,number_of_person,mean_debt
total_income_category,,
A,2.0,8.00
B,354.0,7.06
C,1353.0,8.50
D,21.0,6.02
E,2.0,9.09


**Вывод:** 
По полученным данным нельзя установить *однозначную взаимосвязь между уровнем дохода и способностью вернуть кредит*. Нельзя утверждать, что чем больше доход, тем однозначно больше вероятность вернуть его. Но из полученных данных можно сделать вывод, что в самой низкодоходной категории самый большой процент невозврата долгов.  

Проведем в качестве дополнительного исследования другую категоризацию. Выберем интервалы доходов таким образом, чтобы количество людей в них было примерно одинаковым.

In [30]:
# Для определения границ интервалов для новой категоризации воспользуемся методом qcut
pd.qcut(data['total_income'], 4)

0        (195842.0, 2265604.0]
1         (107507.0, 142594.0]
2         (142594.0, 195842.0]
3        (195842.0, 2265604.0]
4         (142594.0, 195842.0]
                 ...          
21520    (195842.0, 2265604.0]
21521     (142594.0, 195842.0]
21522    (20666.999, 107507.0]
21523    (195842.0, 2265604.0]
21524    (20666.999, 107507.0]
Name: total_income, Length: 21331, dtype: category
Categories (4, interval[float64]): [(20666.999, 107507.0] < (107507.0, 142594.0] < (142594.0, 195842.0] < (195842.0, 2265604.0]]

In [31]:
# Новая функция категоризации данных
def categorize_new_income(income):
    try:
        if 0 <= income < 107507:
            return 'Low'
        elif 107507 <= income < 142594:
            return 'Average_Low'
        elif 142594 <= income < 195842:
            return 'Average_High'
        elif income >= 195842:
            return 'High'
    except:
        pass

In [32]:
# Вводим столбец с новой категоризацией типов дохода
data['total_new_income_category'] = data['total_income'].apply(categorize_new_income)

In [33]:
# Определеяем количество людей и средний процент должников в каждой категории
data.groupby(['total_new_income_category']).apply(summary)

,number_of_person,mean_debt
total_new_income_category,,
Average_High,542.0,8.63
Average_Low,382.0,8.71
High,381.0,7.14
Low,427.0,8.01


**Вывод**: Аналогично, приходим к тому, что по полученным данным *нельзя установить однозначную взаимосвязь между уровнем дохода и способностью вернуть кредит*. 

Для исследования влияния цели кредит на его возрврат срок используем раннее проведенную категоризацию данных и проанализируем количество долгов для групп с разными целями. Для этого опять с группируем данные по 4 представленным категориям. 

In [34]:
# Аналогично прогруппируем данный и посчитаем средний процент должников в каждой категории по цели кредита
data.groupby(['purpose_category']).apply(summary)

,number_of_person,mean_debt
purpose_category,,
операции с автомобилем,400.0,9.35
операции с недвижимостью,780.0,7.26
получение образования,369.0,9.25
проведение свадьбы,183.0,7.91


**Вывод:** 
Из полученных данных видно, что *возврат кредита более вероятен для операций с недвижимостью и проведения свадьбы*. Операции с автомобилем и получения образования имеют больший процент невозвращенных долгов. 

Возможные причины пропусков:


1. В некоторых организациях, рабочих местах может отсутствовать необходимость указывать общий трудовой стаж
2. Часть информации может быть скрыта ввиду государственной тайны. 
3. Частая смена работы некоторыми работниками, может усложнять подсчет общего трудового стажа. 
4. Наличие разных схем и методик подсчета трудового стажа, разные базы данных, затрудняют указание трудового стажа. 
5. Потеря документов - трудовых книжек, трудовых договоров, также может являться причиной пропусков исходных данных. 
6. Желание конфиденциальностью потенциального заемщика. 
7. Невозможность восстановить и собрать необходимые данные ввиду семейных трудностей. 


## Вывод

В ходе исследования влияния различных факторов на факт погашения кредита в срок по статистике о платежеспособности клиентов, можно прийти к следующим выводам: 
однозначной зависимости между **количеством детей** и относительной доли числа должников не установлено. 
Касательно **семейного положения**, можно расположить виды семейные положения упорядоченно по увеличению количества должников в каждой категории: 
        - вдовец / вдова           
        - в разводе                
        - женат / замужем         
        - гражданский брак         
        - Не женат / не замужем  
Откуда видно, что самые благонадежные заемщики - вдовцы и разведенные. 
Однозначной зависимости между **уровнем дохода** и относительной доли должников также не установлено, но в группе с наименьшими доходами можно отметить наибольший процент должников. 
Относительно **целей взятия кредита**, можно расположить их в относительном порядке по увеличению количества должников: 
        - операции с недвижимостью   
        - проведение свадьбы          
        - получение образования       
        - операции с автомобилем  
Откуда видно, что самые возвращаемые цели - операции с недвижимостью и проведение свадьбы. 
